## Chroma Database - Build a Sample Vector DB

In [5]:
from langchain_chroma                           import Chroma
from langchain_community.document_loaders       import TextLoader
from langchain_community.embeddings             import OllamaEmbeddings
from langchain.text_splitter                    import RecursiveCharacterTextSplitter

In [7]:
loader  = TextLoader('data/speech.txt')
data    = loader.load()
data

[Document(metadata={'source': 'data/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairne

In [8]:
textSplitter = RecursiveCharacterTextSplitter(
    chunk_size      = 500,
    chunk_overlap   = 0
)

splits = textSplitter.split_documents(data)

In [10]:
embeddings      = OllamaEmbeddings(model="gemma:2b")

vectorDatabase  = Chroma.from_documents(
    documents   = splits,
    embedding   = embeddings
)
vectorDatabase

## Query The Database

In [11]:
query   = "What does speaker believe is the main reason the US should enter the war?"
result  = vectorDatabase.similarity_search(query)

In [12]:
print(f'First Result:   {result[0].page_content}')

First Result:   for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.


## Saving The Database

In [15]:
vectorDatabase = Chroma.from_documents(
    documents           = splits,
    embedding           = embeddings,
    persist_directory   = "./chromaDB"
)

## Loading The Database

In [16]:
newVectorDatabase = Chroma(
    persist_directory   = "./chromaDB",
    embedding_function  = embeddings
)

In [21]:
result  = newVectorDatabase.similarity_search(query)
score   = newVectorDatabase.similarity_search_with_score(query)

print(f'Similarity Score:   {score}')
print(f'Result:             {result[0].page_content}')

Similarity Score:   [(Document(id='0a28d50c-4d09-4f95-8d8f-e0f75835b651', metadata={'source': 'data/speech.txt'}, page_content='for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'), 3575.0282754688824), (Document(id='cf699634-6f38-4da6-8e0e-9f575d5ca7d3', metadata={'source': 'data/speech.txt'}, page_content='government in the hour of test. They are, most of them, as true and loyal Americans as if they had never known any other fealty or allegiance. They will be prompt to stand with us in rebuking and restraining the few who may be of a different mind and purpose. If there should be disloyalty, it will be dealt with with a firm hand of stern repression; but, if it lifts its head at all, it will lift it only here and there and without countenance excep

## Retriever

In [23]:
retriever = vectorDatabase.as_retriever()
retriever.invoke(query)[0].page_content

'for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'